In [2]:
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [8]:
import json
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification, pipeline, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
import numpy as np
import torch
from evaluate import load as load_metric

model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def load_pubtator(split_name, verbose=False):

    split_to_file = {
        'train': 'Train.PubTator',
        'dev': 'Dev.PubTator',
        'test': 'Test.PubTator'
    }
  
    file_path = f"/home/segr11/INFO_662/project/data/BioRED/{split_to_file[split_name]}"
  
    documents = []
    current_doc = None
    doc_count = 0
  
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
          
            if not line:
                if current_doc and current_doc['text']:
                    documents.append(current_doc)
                    doc_count += 1
                    current_doc = None
                continue
          

            if '|t|' in line or '|a|' in line:
                parts = line.split('|', 2)
                if len(parts) < 3:
                    continue
                pmid = parts[0]
                line_type = parts[1]
                text = parts[2]
              
                if current_doc is None or current_doc['id'] != pmid:
                    current_doc = {
                        'id': pmid,
                        'title': '',
                        'abstract': '',
                        'text': '',
                        'entities': [],
                        'relations': []
                    }
              
                if line_type == 't':
                    current_doc['title'] = text
                elif line_type == 'a':
                    current_doc['abstract'] = text
              
                # Combine title and abstract with space
                current_doc['text'] = (current_doc['title'] + ' ' + current_doc['abstract']).strip()
              
            else:
                # Entity or relation line
                parts = line.split('\t')
              
                if len(parts) >= 6:
                    # Entity annotation
                    pmid, start, end, mention_text, entity_type, identifier = parts[:6]
                  
                    # Filter: only keep Gene, Variant, Disease per BioRED annotation guide
                    # Gene: GeneOrGeneProduct (includes genes, proteins, mRNA, gene products)
                    # Variant: SequenceVariant (genomic/protein variants: substitutions, deletions, insertions)
                    # Disease: DiseaseOrPhenotypicFeature (diseases, symptoms, syndromes, phenotypes)
                    if entity_type in ['GeneOrGeneProduct', 'SequenceVariant', 'DiseaseOrPhenotypicFeature']:
                        entity = {
                            'id': f"T{len(current_doc['entities']) + 1}",
                            'type': entity_type,
                            'text': mention_text,
                            'offsets': [[int(start), int(end)]],
                            'identifier': identifier
                        }
                        current_doc['entities'].append(entity)
                      
                        # Verification: Print first few entities per doc
                        if len(current_doc['entities']) <= 3 and doc_count < 3 and verbose:
                            print(f"  Added entity: '{mention_text}' ({entity_type}) in PMID {pmid}")
                      
                elif len(parts) >= 4:
                    # Relation annotation: PMID RelationType Arg1:ID Arg2:ID [novelty]
                    pmid, rel_type, arg1, arg2 = parts[:4]
                  
                    # Parse arguments (format: "Arg1:MESH:D001249" or just ID)
                    arg1_id = arg1.split(':')[-1] if ':' in arg1 else arg1
                    arg2_id = arg2.split(':')[-1] if ':' in arg2 else arg2
                  
                    # Only keep relevant relation types per BioRED guide for disease-variant/gene
                    # (Association, Positive_Correlation, Negative_Correlation)
                    if rel_type in ['Association', 'Positive_Correlation', 'Negative_Correlation']:
                        relation = {
                            'id': f"R{len(current_doc['relations']) + 1}",
                            'type': rel_type,
                            'arg1_id': arg1_id,
                            'arg2_id': arg2_id
                        }
                        current_doc['relations'].append(relation)
                      
                        # Verification: Print first few relations per doc
                        if len(current_doc['relations']) <= 3 and doc_count < 3 and verbose:
                            print(f"  Added relation: {rel_type} ({arg1_id} -> {arg2_id}) in PMID {pmid}")
      
        # Don't forget last document
        if current_doc and current_doc['text']:
            documents.append(current_doc)
            doc_count += 1
            if doc_count <= 3 and verbose:
                print(f"Loaded final doc {doc_count} (PMID: {current_doc['id']}): {len(current_doc['entities'])} entities, {len(current_doc['relations'])} relations (before filtering)")
  
    print(f"Total docs loaded for {split_name}: {len(documents)}")
    return documents

def filter_relations(doc):
    """
    Filter relations to only include Disease-Variant and Disease-Gene relations.
    Returns filtered document with only relevant relations.
    """
    # Build entity type lookup by identifier
    entity_info = {}
    for entity in doc['entities']:
        entity_info[entity['identifier']] = entity['type']
  
    filtered_relations = []
    for relation in doc['relations']:
        arg1_id = relation['arg1_id']
        arg2_id = relation['arg2_id']
      
        # Get entity types
        type1 = entity_info.get(arg1_id, None)
        type2 = entity_info.get(arg2_id, None)
      
        if type1 and type2:
            # Check if it's a Disease-Variant or Disease-Gene relation
            entity_pair = {type1, type2}
          
            if entity_pair == {'DiseaseOrPhenotypicFeature', 'SequenceVariant'}:
                # Disease-Variant relation
                filtered_relations.append(relation)
                # Verification: Print kept relations
                if len(filtered_relations) <= 3:
                    print(f"  Kept Disease-Variant: {relation['type']} ({arg1_id} -> {arg2_id}) in PMID {doc['id']}")
            elif entity_pair == {'DiseaseOrPhenotypicFeature', 'GeneOrGeneProduct'}:
                # Disease-Gene relation
                filtered_relations.append(relation)
                # Verification: Print kept relations
                if len(filtered_relations) <= 3:
                    print(f"  Kept Disease-Gene: {relation['type']} ({arg1_id} -> {arg2_id}) in PMID {doc['id']}")
  
    # Create filtered document
    filtered_doc = doc.copy()
    filtered_doc['relations'] = filtered_relations
  
    # Verification: Print summary for first few docs
    if len(doc['entities']) > 0:  # Only for non-empty
        print(f"Filtered doc {doc['id']}: {len(doc['entities'])} entities, {len(filtered_relations)} relations (after filter)")
  
    return filtered_doc

# Load and filter data
if __name__ == "__main__":
    print("Loading BioRED data in PubTator format...")
    raw_data = {
        "train": [filter_relations(doc) for doc in load_pubtator("train")],
        "validation": [filter_relations(doc) for doc in load_pubtator("dev")],
        "test": [filter_relations(doc) for doc in load_pubtator("test")]
    }
    print("✓ Successfully loaded and filtered all splits\n")

# Define filtered labels
ner_labels = ["O", "B-GENE", "I-GENE", "B-VARIANT", "I-VARIANT", "B-DISEASE", "I-DISEASE"]
ner_id2label = {i: label for i, label in enumerate(ner_labels)}
ner_label2id = {v: k for k, v in ner_id2label.items()}
num_ner_labels = len(ner_labels)

# Relation types from BioRED guidelines for Disease-Variant and Disease-Gene
re_labels = [
    "NoRel",
    "Positive_Correlation",
    "Negative_Correlation",
    "Association"
]
re_id2label = {i: label for i, label in enumerate(re_labels)}
re_label2id = {v: k for k, v in re_id2label.items()}
num_re_labels = len(re_labels)

# Entity type mapping for easier reference (per BioRED guide)
ENTITY_TYPE_MAP = {
    'GeneOrGeneProduct': 'GENE',      # Genes, proteins, mRNA, gene products
    'SequenceVariant': 'VARIANT',     # Genomic/protein variants (substitutions, deletions, insertions)
    'DiseaseOrPhenotypicFeature': 'DISEASE'  # Diseases, symptoms, syndromes, phenotypes
}

def tokenize_and_align_labels(text, entities, tokenizer):
    encoding = tokenizer(text, return_offsets_mapping=True, truncation=True, padding='max_length', max_length=512)
    offset_mapping = encoding['offset_mapping']
    labels = [ner_label2id["O"]] * len(offset_mapping)
   
    for entity in entities:
        entity_type = ENTITY_TYPE_MAP.get(entity['type'])
        if not entity_type:
            continue
        for offset in entity['offsets']:
            start, end = offset[0], offset[1]
            token_start_idx = None
            token_end_idx = None
            for idx, (token_start, token_end) in enumerate(offset_mapping):
                if token_start == token_end:  # Skip subwords with no span
                    continue
                if token_start <= start < token_end:
                    token_start_idx = idx
                if token_start < end <= token_end:
                    token_end_idx = idx
                    break
            if token_start_idx is not None:
                b_label = f"B-{entity_type}"
                if b_label in ner_label2id:
                    labels[token_start_idx] = ner_label2id[b_label]
                if token_end_idx is not None:
                    i_label = f"I-{entity_type}"
                    if i_label in ner_label2id:
                        for idx in range(token_start_idx + 1, token_end_idx + 1):
                            labels[idx] = ner_label2id[i_label]
   
    return encoding, labels

def extract_filtered_relation_pairs(doc):
    entities = doc['entities']
    relations = doc['relations']
    entity_dict = {e['identifier']: e for e in entities}
    relation_dict = {}
    for rel in relations:
        key = (rel['arg1_id'], rel['arg2_id'])
        relation_dict[key] = rel['type']
        key_reverse = (rel['arg2_id'], rel['arg1_id'])
        relation_dict[key_reverse] = rel['type']
   
    pairs = []
    for i, e1 in enumerate(entities):
        for e2 in entities[i+1:]:
            type_pair = {e1['type'], e2['type']}
            valid_pairs = [
                {'DiseaseOrPhenotypicFeature', 'SequenceVariant'},
                {'DiseaseOrPhenotypicFeature', 'GeneOrGeneProduct'}
            ]
            if type_pair not in valid_pairs:
                continue
            key = (e1['identifier'], e2['identifier'])
            rel_type = relation_dict.get(key, 'NoRel')
            rel_label = re_label2id.get(rel_type, re_label2id["NoRel"])
            pairs.append({
                'entity1': e1,
                'entity2': e2,
                'relation_label': rel_label,
                'relation_type': rel_type
            })
    return pairs

# Create datasets (focus on train split, but include dev for eval)
print("Creating datasets...")
ner_data = {'train': [], 'validation': []}
re_data = {'train': [], 'validation': []}

# Train split
for doc in raw_data["train"]:
    text = doc['text']
    # NER
    encoding, labels = tokenize_and_align_labels(text, doc['entities'], tokenizer)
    ner_data['train'].append({
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': labels,
        'length': len(labels)
    })
    # RE
    pairs = extract_filtered_relation_pairs(doc)
    for pair in pairs:
        e1_text = pair['entity1']['text']
        e2_text = pair['entity2']['text']
        e1_type = ENTITY_TYPE_MAP.get(pair['entity1']['type'], '')
        e2_type = ENTITY_TYPE_MAP.get(pair['entity2']['type'], '')
        marked_text = f"[{e1_type}] {e1_text} [/{e1_type}] and [{e2_type}] {e2_text} [/{e2_type}] in context: {text}"
        encoding = tokenizer(marked_text, truncation=True, max_length=512, padding=False)
        re_data['train'].append({
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'labels': pair['relation_label'],
            'length': len(encoding['input_ids'])
        })

# Dev split for validation (subset for speed, or full)
for doc in raw_data["validation"][:100]:  # Limit for demo; use full for real
    text = doc['text']
    # NER
    encoding, labels = tokenize_and_align_labels(text, doc['entities'], tokenizer)
    ner_data['validation'].append({
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': labels,
        'length': len(labels)
    })
    # RE
    pairs = extract_filtered_relation_pairs(doc)
    for pair in pairs:
        e1_text = pair['entity1']['text']
        e2_text = pair['entity2']['text']
        e1_type = ENTITY_TYPE_MAP.get(pair['entity1']['type'], '')
        e2_type = ENTITY_TYPE_MAP.get(pair['entity2']['type'], '')
        marked_text = f"[{e1_type}] {e1_text} [/{e1_type}] and [{e2_type}] {e2_text} [/{e2_type}] in context: {text}"
        encoding = tokenizer(marked_text, truncation=True, max_length=512, padding=False)
        re_data['validation'].append({
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'labels': pair['relation_label'],
            'length': len(encoding['input_ids'])
        })

# Convert to Datasets
ner_dataset = DatasetDict({
    'train': Dataset.from_list(ner_data['train']),
    'validation': Dataset.from_list(ner_data['validation'])
})
re_dataset = DatasetDict({
    'train': Dataset.from_list(re_data['train']),
    'validation': Dataset.from_list(re_data['validation'])
})

print(f"NER Dataset sizes: Train={len(ner_dataset['train'])}, Val={len(ner_dataset['validation'])}")
print(f"RE Dataset sizes: Train={len(re_dataset['train'])}, Val={len(re_dataset['validation'])}")

Loading BioRED data in PubTator format...
Total docs loaded for train: 400
  Kept Disease-Gene: Association (3175 -> D003924) in PMID 10491763
Filtered doc 10491763: 23 entities, 1 relations (after filter)
Filtered doc 10661407: 9 entities, 0 relations (after filter)
  Kept Disease-Variant: Positive_Correlation (D001943 -> c|INS|5382|C) in PMID 10788334
  Kept Disease-Variant: Positive_Correlation (D001943 -> c|DEL|4153|A) in PMID 10788334
  Kept Disease-Variant: Positive_Correlation (D001943 -> rs28897672) in PMID 10788334
Filtered doc 10788334: 23 entities, 12 relations (after filter)
Filtered doc 11009181: 6 entities, 0 relations (after filter)
  Kept Disease-Gene: Association (D014594 -> 84084) in PMID 11054569
Filtered doc 11054569: 10 entities, 1 relations (after filter)
Filtered doc 11773892: 35 entities, 0 relations (after filter)
  Kept Disease-Gene: Association (6647 -> D000690) in PMID 12442272
  Kept Disease-Variant: Negative_Correlation (rs80265967 -> D000690) in PMID 1244

NER Dataset sizes: Train=400, Val=100
RE Dataset sizes: Train=46680, Val=13032


In [ ]:
import pandas as pd

def export_re_data(dataset_split, output_file):
    data = []
    for item in dataset_split:
        # Decode input_ids back to text (approx; use your marked_text logic)
        decoded = tokenizer.decode(item['input_ids'], skip_special_tokens=True)
        label_str = re_id2label[item['labels']]
        data.append({'sentence': decoded, 'label': 1 if label_str != "NoRel" else 0})  # Binary for repo demo; adjust for multi-class
    pd.DataFrame(data).to_csv(output_file, sep='\t', index=False)

export_re_data(re_dataset["train"], "train_re.tsv")
export_re_data(re_dataset["validation"], "dev_re.tsv")  # Repo uses 'dev'

In [19]:
# Fine-tune RE (Sequence Classification)
print("\n=== Fine-tuning RE ===")
re_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_re_labels,
    id2label=re_id2label,
    label2id=re_label2id
)

# Metrics for RE
try:
    re_metric = evaluate.load("accuracy")
except Exception as e:
    print(f"Warning: evaluate.load failed ({e}). Falling back to datasets metric.")
    from datasets import load_metric
    re_metric = load_metric("accuracy")

def compute_re_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return re_metric.compute(predictions=predictions, references=labels)

re_args = TrainingArguments(
    output_dir="./biobert-re-finetuned",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to=None
)

re_trainer = Trainer(
    model=re_model,
    args=re_args,
    train_dataset=re_dataset['train'],
    eval_dataset=re_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_re_metrics,
)

print("Starting RE training...")
re_trainer.train()
re_trainer.save_model("./biobert-re-finetuned")
print("RE fine-tuning complete!")


=== Fine-tuning RE ===


Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

NameError: name 'PartialState' is not defined

In [12]:
!pip install --upgrade accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
  Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.7 MB/s  0:00:00
Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (485 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [accelerate]7 [accelerate]_hub]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.29.2 requires huggingface-hub<1.0,>=0.14.1, but y

In [16]:
!pip install --upgrade pip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.1 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2


In [17]:
!pip install --upgrade transformers evaluate datasets torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-4.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached pyarrow-22.0.0.tar.gz (1.2 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × installing build dependencies for pyarrow did not run successfully.
  │ ex

In [ ]:
# Inference Pipeline Example
print("\n=== Inference Example ===")
ner_pipe = pipeline("ner", model="./biobert-ner-finetuned", tokenizer=tokenizer, aggregation_strategy="simple")
re_pipe = pipeline("text-classification", model="./biobert-re-finetuned", tokenizer=tokenizer)

def end_to_end_extract(text):
    entities = ner_pipe(text)
    genes = [e for e in entities if "GENE" in e["entity_group"]]
    variants = [e for e in entities if "VARIANT" in e["entity_group"]]
    diseases = [e for e in entities if "DISEASE" in e["entity_group"]]
    
    relations = []
    # Example: Disease-Gene pairs
    for disease in diseases:
        for gene in genes:
            pair_text = f"[DISEASE] {disease['word']} [/DISEASE] and [GENE] {gene['word']} [/GENE] in context: {text}"
            rel = re_pipe(pair_text)[0]
            relations.append({
                "disease": disease["word"],
                "gene": gene["word"],
                "relation": rel["label"],
                "score": rel["score"]
            })
    return entities, relations

In [ ]:
# Test on sample text (e.g., from raw_data["train"][0])
sample_text = raw_data["train"][0]['text'][:500]  # Truncate for demo
entities, relations = end_to_end_extract(sample_text)
print("Sample Entities:", entities[:3])
print("Sample Relations:", relations[:3] if relations else "No relations found")